In [16]:
#Must run atleast once!
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


import nltk
nltk.download('averaged_perceptron_tagger')
'''
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
'''

import contractions

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS

import gensim





[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\General IRIS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# DATA CLEANING ENGLISH


In [17]:
df = pd.read_csv('2.1PreprocessedMalayUpdatedDec19.csv', encoding='utf-8')
df.head(50)

,Username,Userid,Rating,Helpfulness,Comment,TimeCreated,Target,CommentToStr,SimpleCommentManglish,LengthManglish,TotalEmoji,TotalPunct,TotalCaps,CommentClean1SimpleSpellCorrector,CommentClean2Normalizing,CommentClean4EnglishComment
0,i*****_,218195265,3,0.0,SSD boleh pakai cuma...bila dah transfer tetib...,2/25/2022 22:20,1,SSD boleh pakai cuma...bila dah transfer tetib...,solid state drive boleh pakai cuma bila dah tr...,23,0,12,3,solid state drive boleh pakai cuma bila dah pi...,solid state drive boleh pakai cuma bila dah pi...,solid state drive can only be used when moving...
1,simonchan1905,705330191,1,1.0,Never trust or purchase usb or sd card if the ...,2022-07-29 08:29:09,1,Never trust or purchase usb or sd card if the ...,never trust or purchase USB or sd card if the ...,42,0,2,7,never trust atau purchase USB atau sd card jik...,never trust atau purchase USB atau sd card jik...,i never trust or buy usb or sd card if the pri...
2,ladydelilah,90528909,5,0.0,"Alhamdulillah dh smpi, terus guna.. saje amek ...",2022-01-07 11:28:36,0,"Alhamdulillah dh smpi, terus guna.. saje amek ...",alhamdulillah dh sampai terus guna saje amek k...,14,0,5,1,alhamdulillah dah sampai terus guna sahaja amb...,alhamdulillah dah sampai terus guna sahaja amb...,thank god it's just to take pink kaler fans so...
3,anna_aizad,34734312,1,7.0,All my file corrupted after saving a few file ...,2022-07-13 14:28:30,1,All my file corrupted after saving a few file ...,all saya punya file corrupted after saving a f...,30,0,0,1,semua saya punya file corrupted after saving a...,semua saya punya file corrupted after saving a...,all i have corrupted file after saving a few f...
4,b*****d,758613696,5,2.0,"Box came bubble-wrapped, so everything is in g...",2022-07-12 13:33:12,0,"Box came bubble-wrapped, so everything is in g...",box came bubble wrapped so everything ialah da...,44,0,10,7,box came bubble wrapped jadi semua ialah dalam...,box came bubble wrapped jadi semua ialah dalam...,the box came bubble wrapped so all was in good...
5,beskotkebeng,184848780,5,2.0,Copy speed based on your internal storage. Man...,2020-10-04 09:57:55,1,Copy speed based on your internal storage. Man...,copy speed based your internal storage manage ...,47,1,6,7,copy speed based awak internal storage manage ...,copy speed based awak internal storage manage ...,your internal speed copy manages to get megaby...
6,mikrobistro,554068149,2,3.0,Barang sampai dlm keadaan baik tp kenapa hard ...,5/19/2022 12:27,1,Barang sampai dlm keadaan baik tp kenapa hard ...,barang sampai dlm keadaan baik tp kenapa hard ...,20,0,6,1,barang sampai dalam keadaan baik tetapi kenapa...,barang sampai dalam keadaan baik tetapi kenapa...,goods are in good condition but why is it hard...
7,suraya0806,30636902,5,0.0,Received item in good condition. Secure packag...,2021-08-18 19:00:41,0,Received item in good condition. Secure packag...,received item dalam good condition secure pack...,10,0,2,3,received item dalam baik condition secure pack...,received item dalam baik condition secure pack...,"received an item in good condition, secure the..."
8,aimestudio,39266309,5,0.0,Good Product Quality Seagate ✅\nLaju. if nak s...,2021-06-28 03:17:32,1,Good Product Quality Seagate ✅\nLaju. if nak s...,good product quality SEAGATE laju if nak simpa...,53,4,5,26,baik product kualiti SEAGATE laju jika nak sim...,baik product kualiti SEAGATE laju jika nak sim...,seagate quality product is quick to keep the m...
9,j*****n,219751578,5,0.0,Item received in good condition but yet to be ...,2021-06-22 20:41:30,0,Item received in good condition but yet to be ...,item received dalam good condition but yet to ...,11,20,1,1,item received dalam baik condition tetapi yet ...,item received dalam baik condition tetapi yet ...,the item received is in good condition but is ...


# REMOVES ANY NULLS

In [18]:
print('Before : ',df.shape[0], '\n')
print(df.isnull().sum(), '\n')
df = df.dropna()
print('After : ',df.shape[0], '\n')
print(df.isnull().sum())

Before :  1171 

Username                             0
Userid                               0
Rating                               0
Helpfulness                          0
Comment                              0
TimeCreated                          0
Target                               0
CommentToStr                         0
SimpleCommentManglish                0
LengthManglish                       0
TotalEmoji                           0
TotalPunct                           0
TotalCaps                            0
CommentClean1SimpleSpellCorrector    9
CommentClean2Normalizing             9
CommentClean4EnglishComment          9
dtype: int64 

After :  1162 

Username                             0
Userid                               0
Rating                               0
Helpfulness                          0
Comment                              0
TimeCreated                          0
Target                               0
CommentToStr                         0
SimpleCommentMan

## Re-To Lower Comments

In [19]:
df['CommentClean4EnglishComment'] = df['CommentClean4EnglishComment'].apply(lambda x: x.lower())

## 1. Tokenizing Words

In [20]:
#Token* ONLY ~TEST
"""
def text_tokenizer(text):
    if text:
        result = []
        for token in gensim.utils.simple_preprocess(text):
            result.append(token)
        return result
    return ''
"""

df['EnglishTokenized'] = df['CommentClean4EnglishComment'].fillna('').astype(str).apply(lambda x : x.split())

# 2.Expand Contraction

In [21]:
def expand(text):
    if text:
        expanded = []
        for word in text:
            expanded.append(contractions.fix(word))
        newExpanded = ' '.join(expanded)

        newTokenized = newExpanded.split()

        return newTokenized
    return ''

df['EnglishExpandContraction'] = df['EnglishTokenized'].apply(lambda x : expand(x))

[Feature Engineering Part 2 Final]
## 3.Part Of Speech Tagging [POS Tag]



### 2.1 Collect Tags

In [22]:
def pos_taggings (row):

    if row:
        postags = nltk.pos_tag(row)
        classes = list()
        for abs in postags:
            #print(abs)
            classes.append(abs[1])
        return classes
    return ''

df['PosTags'] = df['EnglishExpandContraction'].apply(lambda x: pos_taggings(x))

### 2.2 Group & Count Tags [Verbs, Adverbs, Adjectives, Nouns]

In [23]:
def find_no_class(count, class_name = ""):

    total = 0
    for key in count.keys():

        if key.startswith(class_name):

            total += count[key]

    return total

def get_classes(row, grammatical_class = ""):

    if row:
        count = Counter(row)
        return find_no_class(count, class_name = grammatical_class)/len(row)

    return 0

In [24]:
df["FrequencyAdverbs"] = df['PosTags'].apply(lambda x: get_classes(x, "RB"))
df["FrequencyVerbs"] = df['PosTags'].apply(lambda x: get_classes(x, "VB"))
df["FrequencyAdjectives"] = df['PosTags'].apply(lambda x: get_classes(x, "JJ"))
df["FrequencyNouns"] = df['PosTags'].apply(lambda x: get_classes(x, "NN"))

In [25]:
df.head(20)

,Username,Userid,Rating,Helpfulness,Comment,TimeCreated,Target,CommentToStr,SimpleCommentManglish,LengthManglish,...,CommentClean1SimpleSpellCorrector,CommentClean2Normalizing,CommentClean4EnglishComment,EnglishTokenized,EnglishExpandContraction,PosTags,FrequencyAdverbs,FrequencyVerbs,FrequencyAdjectives,FrequencyNouns
0,i*****_,218195265,3,0.0,SSD boleh pakai cuma...bila dah transfer tetib...,2/25/2022 22:20,1,SSD boleh pakai cuma...bila dah transfer tetib...,solid state drive boleh pakai cuma bila dah tr...,23,...,solid state drive boleh pakai cuma bila dah pi...,solid state drive boleh pakai cuma bila dah pi...,solid state drive can only be used when moving...,"[solid, state, drive, can, only, be, used, whe...","[solid, state, drive, can, only, be, used, whe...","[JJ, NN, NN, MD, RB, VB, VBN, WRB, VBG, DT, NN...",0.090909,0.181818,0.090909,0.318182
1,simonchan1905,705330191,1,1.0,Never trust or purchase usb or sd card if the ...,2022-07-29 08:29:09,1,Never trust or purchase usb or sd card if the ...,never trust or purchase USB or sd card if the ...,42,...,never trust atau purchase USB atau sd card jik...,never trust atau purchase USB atau sd card jik...,i never trust or buy usb or sd card if the pri...,"[i, never, trust, or, buy, usb, or, sd, card, ...","[i, never, trust, or, buy, usb, or, sd, card, ...","[NN, RB, NN, CC, VB, JJ, CC, JJ, NN, IN, DT, N...",0.088889,0.200000,0.177778,0.200000
2,ladydelilah,90528909,5,0.0,"Alhamdulillah dh smpi, terus guna.. saje amek ...",2022-01-07 11:28:36,0,"Alhamdulillah dh smpi, terus guna.. saje amek ...",alhamdulillah dh sampai terus guna saje amek k...,14,...,alhamdulillah dah sampai terus guna sahaja amb...,alhamdulillah dah sampai terus guna sahaja amb...,thank god it's just to take pink kaler fans so...,"[thank, god, it's, just, to, take, pink, kaler...","[thank, god, it, is, just, to, take, pink, kal...","[NN, NN, PRP, VBZ, RB, TO, VB, NN, NN, NNS, IN...",0.071429,0.214286,0.071429,0.357143
3,anna_aizad,34734312,1,7.0,All my file corrupted after saving a few file ...,2022-07-13 14:28:30,1,All my file corrupted after saving a few file ...,all saya punya file corrupted after saving a f...,30,...,semua saya punya file corrupted after saving a...,semua saya punya file corrupted after saving a...,all i have corrupted file after saving a few f...,"[all, i, have, corrupted, file, after, saving,...","[all, i, have, corrupted, file, after, saving,...","[DT, NNS, VBP, VBN, NN, IN, VBG, DT, JJ, NNS, ...",0.064516,0.258065,0.129032,0.290323
4,b*****d,758613696,5,2.0,"Box came bubble-wrapped, so everything is in g...",2022-07-12 13:33:12,0,"Box came bubble-wrapped, so everything is in g...",box came bubble wrapped so everything ialah da...,44,...,box came bubble wrapped jadi semua ialah dalam...,box came bubble wrapped jadi semua ialah dalam...,the box came bubble wrapped so all was in good...,"[the, box, came, bubble, wrapped, so, all, was...","[the, box, came, bubble, wrapped, so, all, was...","[DT, NN, VBD, JJ, VBN, RB, DT, VBD, IN, JJ, NN...",0.073171,0.195122,0.268293,0.243902
5,beskotkebeng,184848780,5,2.0,Copy speed based on your internal storage. Man...,2020-10-04 09:57:55,1,Copy speed based on your internal storage. Man...,copy speed based your internal storage manage ...,47,...,copy speed based awak internal storage manage ...,copy speed based awak internal storage manage ...,your internal speed copy manages to get megaby...,"[your, internal, speed, copy, manages, to, get...","[your, internal, speed, copy, manages, to, get...","[PRP$, JJ, NN, NN, VBZ, TO, VB, NNS, TO, VB, J...",0.000000,0.285714,0.142857,0.285714
6,mikrobistro,554068149,2,3.0,Barang sampai dlm keadaan baik tp kenapa hard ...,5/19/2022 12:27,1,Barang sampai dlm keadaan baik tp kenapa hard ...,barang sampai dlm keadaan baik tp kenapa hard ...,20,...,barang sampai dalam keadaan baik tetapi kenapa...,barang sampai dalam keadaan baik tetapi kenapa...,goods are in good condition but why is it hard...,"[goods, are, in, good

## 3. Removing Stop Words

In [26]:
stop = set(stopwords.words('english'))


def remove_stopwords(tags,tokens):

    final_text = []
    latestTags = []

    for i in range (len(tokens)):
        if tokens[i] not in stop:
            final_text.append(tokens[i])
            latestTags.append(tags[i])
    return pd.Series([final_text,latestTags])


def exeStopWordsRemover(df):
    df[['StopWords','LatestPosTagForLemmatize']] = df.apply(lambda x: remove_stopwords(x['PosTags'],x['EnglishExpandContraction']), axis=1)
    return df

df = exeStopWordsRemover(df)

In [27]:
df.head(20)

,Username,Userid,Rating,Helpfulness,Comment,TimeCreated,Target,CommentToStr,SimpleCommentManglish,LengthManglish,...,CommentClean4EnglishComment,EnglishTokenized,EnglishExpandContraction,PosTags,FrequencyAdverbs,FrequencyVerbs,FrequencyAdjectives,FrequencyNouns,StopWords,LatestPosTagForLemmatize
0,i*****_,218195265,3,0.0,SSD boleh pakai cuma...bila dah transfer tetib...,2/25/2022 22:20,1,SSD boleh pakai cuma...bila dah transfer tetib...,solid state drive boleh pakai cuma bila dah tr...,23,...,solid state drive can only be used when moving...,"[solid, state, drive, can, only, be, used, whe...","[solid, state, drive, can, only, be, used, whe...","[JJ, NN, NN, MD, RB, VB, VBN, WRB, VBG, DT, NN...",0.090909,0.181818,0.090909,0.318182,"[solid, state, drive, used, moving, file, empt...","[JJ, NN, NN, VBN, VBG, NN, JJ, NN, NN, NN, VB,..."
1,simonchan1905,705330191,1,1.0,Never trust or purchase usb or sd card if the ...,2022-07-29 08:29:09,1,Never trust or purchase usb or sd card if the ...,never trust or purchase USB or sd card if the ...,42,...,i never trust or buy usb or sd card if the pri...,"[i, never, trust, or, buy, usb, or, sd, card, ...","[i, never, trust, or, buy, usb, or, sd, card, ...","[NN, RB, NN, CC, VB, JJ, CC, JJ, NN, IN, DT, N...",0.088889,0.200000,0.177778,0.200000,"[never, trust, buy, usb, sd, card, price, good...","[RB, NN, VB, JJ, JJ, NN, NN, JJ, JJ, VBD, JJ, ..."
2,ladydelilah,90528909,5,0.0,"Alhamdulillah dh smpi, terus guna.. saje amek ...",2022-01-07 11:28:36,0,"Alhamdulillah dh smpi, terus guna.. saje amek ...",alhamdulillah dh sampai terus guna saje amek k...,14,...,thank god it's just to take pink kaler fans so...,"[thank, god, it's, just, to, take, pink, kaler...","[thank, god, it, is, just, to, take, pink, kal...","[NN, NN, PRP, VBZ, RB, TO, VB, NN, NN, NNS, IN...",0.071429,0.214286,0.071429,0.357143,"[thank, god, take, pink, kaler, fans, pretty.]","[NN, NN, VB, NN, NN, NNS, JJ]"
3,anna_aizad,34734312,1,7.0,All my file corrupted after saving a few file ...,2022-07-13 14:28:30,1,All my file corrupted after saving a few file ...,all saya punya file corrupted after saving a f...,30,...,all i have corrupted file after saving a few f...,"[all, i, have, corrupted, file, after, saving,...","[all, i, have, corrupted, file, after, saving,...","[DT, NNS, VBP, VBN, NN, IN, VBG, DT, JJ, NNS, ...",0.064516,0.258065,0.129032,0.290323,"[corrupted, file, saving, files, even, though,...","[VBN, NN, VBG, NNS, RB, IN, VBP, JJ, NN, NN, N..."
4,b*****d,758613696,5,2.0,"Box came bubble-wrapped, so everything is in g...",2022-07-12 13:33:12,0,"Box came bubble-wrapped, so everything is in g...",box came bubble wrapped so everything ialah da...,44,...,the box came bubble wrapped so all was in good...,"[the, box, came, bubble, wrapped, so, all, was...","[the, box, came, bubble, wrapped, so, all, was...","[DT, NN, VBD, JJ, VBN, RB, DT, VBD, IN, JJ, NN...",0.073171,0.195122,0.268293,0.243902,"[box, came, bubble, wrapped, good, condition, ...","[NN, VBD, JJ, VBN, JJ, NN, NN, JJ, NN, NN, JJ,..."
5,beskotkebeng,184848780,5,2.0,Copy speed based on your internal storage. Man...,2020-10-04 09:57:55,1,Copy speed based on your internal storage. Man...,copy speed based your internal storage manage ...,47,...,your internal speed copy manages to get megaby...,"[your, internal, speed, copy, manages, to, get...","[your, internal, speed, copy, manages, to, get...","[PRP$, JJ, NN, NN, VBZ, TO, VB, NNS, TO, VB, J...",0.000000,0.285714,0.142857,0.285714,"[internal, speed, copy, manages, get, megabyte...","[JJ, NN, NN, VBZ, VB, NNS, VB, JJ, NN, VB]"
6,mikrobistro,554068149,2,3.0,Barang sampai dlm keadaan baik tp kenapa hard ...,5/19/2022 12:27,1,Barang sampai dlm keadaan baik tp kenapa hard ...,barang sampai dlm keadaan baik tp kenapa hard ...,20,...,goods are in good condition but why is it hard...,"[goods, are, in, good, condition, but, why, is...","[goods, are, in, good, condition, but, why, is...","[NNS, VBP, IN, JJ, NN, CC, WRB, VBZ, PRP, JJ, .

stop = set(stopwords.words('english'))

def remove_stopwords(text):

    final_text = []
    #text = str(text).lower()
    textCount = 0
    for i in text:
        if i not in stop:
            final_text.append(i)
        textCount += 1
    return final_text

def clean_text(text):
    text = remove_stopwords(text)
    return text

def exeStopWordsRemover(df):
    df['StopWords'] = df['EnglishTokenized'].apply(lambda x: clean_text(x))
    return df

df = exeStopWordsRemover(df)


## 4. Lemmatization

In [28]:
def lematize(stopwords,latestpostags,lemmatizer):


    result = []
    # Length stopwords & latestpostags are ==

    for i in range (len(stopwords)):

        foundtag = 'n'

        #If latestpostag found in selected then use it else use default


        if latestpostags[i][0:2] == 'NN':
            foundtag = 'n'

        elif latestpostags[i][0:2] == 'RB':
            foundtag = 'r'

        elif latestpostags[i][0:2] == 'VB':
            foundtag = 'v'
        else:
            foundtag = 'a'

        result.append(lemmatizer.lemmatize(stopwords[i], pos= foundtag))

    return result

df['Lematize'] = df.apply(lambda x : lematize(x['StopWords'],x['LatestPosTagForLemmatize'],WordNetLemmatizer()), axis= 1)
df['FinalCleanComment'] = df['Lematize'].apply(lambda x : ' '.join(x))

def lematize(aCleanList,lemmatizer):

    result = []

    for i, val in enumerate (aCleanList):
        result.append(lemmatizer.lemmatize(val))
    return result

df['FinalCleanComment'] = df['StopWords'].apply(lambda x : lematize(x,WordNetLemmatizer()))
df['FinalCleanComment'] = df['FinalCleanComment'].apply(lambda x : ' '.join(x))

In [29]:
df.head(40)

,Username,Userid,Rating,Helpfulness,Comment,TimeCreated,Target,CommentToStr,SimpleCommentManglish,LengthManglish,...,EnglishExpandContraction,PosTags,FrequencyAdverbs,FrequencyVerbs,FrequencyAdjectives,FrequencyNouns,StopWords,LatestPosTagForLemmatize,Lematize,FinalCleanComment
0,i*****_,218195265,3,0.0,SSD boleh pakai cuma...bila dah transfer tetib...,2/25/2022 22:20,1,SSD boleh pakai cuma...bila dah transfer tetib...,solid state drive boleh pakai cuma bila dah tr...,23,...,"[solid, state, drive, can, only, be, used, whe...","[JJ, NN, NN, MD, RB, VB, VBN, WRB, VBG, DT, NN...",0.090909,0.181818,0.090909,0.318182,"[solid, state, drive, used, moving, file, empt...","[JJ, NN, NN, VBN, VBG, NN, JJ, NN, NN, NN, VB,...","[solid, state, drive, use, move, file, empty, ...",solid state drive use move file empty waste ti...
1,simonchan1905,705330191,1,1.0,Never trust or purchase usb or sd card if the ...,2022-07-29 08:29:09,1,Never trust or purchase usb or sd card if the ...,never trust or purchase USB or sd card if the ...,42,...,"[i, never, trust, or, buy, usb, or, sd, card, ...","[NN, RB, NN, CC, VB, JJ, CC, JJ, NN, IN, DT, N...",0.088889,0.200000,0.177778,0.200000,"[never, trust, buy, usb, sd, card, price, good...","[RB, NN, VB, JJ, JJ, NN, NN, JJ, JJ, VBD, JJ, ...","[never, trust, buy, usb, sd, card, price, good...",never trust buy usb sd card price good true le...
2,ladydelilah,90528909,5,0.0,"Alhamdulillah dh smpi, terus guna.. saje amek ...",2022-01-07 11:28:36,0,"Alhamdulillah dh smpi, terus guna.. saje amek ...",alhamdulillah dh sampai terus guna saje amek k...,14,...,"[thank, god, it, is, just, to, take, pink, kal...","[NN, NN, PRP, VBZ, RB, TO, VB, NN, NN, NNS, IN...",0.071429,0.214286,0.071429,0.357143,"[thank, god, take, pink, kaler, fans, pretty.]","[NN, NN, VB, NN, NN, NNS, JJ]","[thank, god, take, pink, kaler, fan, pretty.]",thank god take pink kaler fan pretty.
3,anna_aizad,34734312,1,7.0,All my file corrupted after saving a few file ...,2022-07-13 14:28:30,1,All my file corrupted after saving a few file ...,all saya punya file corrupted after saving a f...,30,...,"[all, i, have, corrupted, file, after, saving,...","[DT, NNS, VBP, VBN, NN, IN, VBG, DT, JJ, NNS, ...",0.064516,0.258065,0.129032,0.290323,"[corrupted, file, saving, files, even, though,...","[VBN, NN, VBG, NNS, RB, IN, VBP, JJ, NN, NN, N...","[corrupt, file, save, file, even, though, buy,...",corrupt file save file even though buy solid s...
4,b*****d,758613696,5,2.0,"Box came bubble-wrapped, so everything is in g...",2022-07-12 13:33:12,0,"Box came bubble-wrapped, so everything is in g...",box came bubble wrapped so everything ialah da...,44,...,"[the, box, came, bubble, wrapped, so, all, was...","[DT, NN, VBD, JJ, VBN, RB, DT, VBD, IN, JJ, NN...",0.073171,0.195122,0.268293,0.243902,"[box, came, bubble, wrapped, good, condition, ...","[NN, VBD, JJ, VBN, JJ, NN, NN, JJ, NN, NN, JJ,...","[box, come, bubble, wrap, good, condition, cap...",box come bubble wrap good condition capacity t...
5,beskotkebeng,184848780,5,2.0,Copy speed based on your internal storage. Man...,2020-10-04 09:57:55,1,Copy speed based on your internal storage. Man...,copy speed based your internal storage manage ...,47,...,"[your, internal, speed, copy, manages, to, get...","[PRP$, JJ, NN, NN, VBZ, TO, VB, NNS, TO, VB, J...",0.000000,0.285714,0.142857,0.285714,"[internal, speed, copy, manages, get, megabyte...","[JJ, NN, NN, VBZ, VB, NNS, VB, JJ, NN, VB]","[internal, speed, copy, manage, get, megabyte,...",internal speed copy manage get megabyte write ...
6,mikrobistro,554068149,2,3.0,Barang sampai dlm keadaan baik tp kenapa hard ...,5/19/2022 12:27,1,Barang sampai dlm keadaan baik tp kenapa hard ...,barang sampai dlm keadaan baik tp kenapa hard ...,20,...,"[goods, are, in, good, condition, but, why, is...","[NNS, VBP, IN, JJ, NN, CC, WRB, VBZ, PRP, JJ, ...",0.000000,0.240000,0.120000,0.240000,"[goods, good, condition, hard, disc, cannot, c...","[NNS, JJ, NN, JJ, VB, VBP, VB, NN, JJ, NN, 

# WORD CLOUDS

#Use for True
"""cloud = WordCloud(max_words = 18000,
                  width = 1600,
                  height = 800,
                  stopwords = STOPWORDS,
                  background_color = "white",
                  collocations=False).generate(" ".join(df["FinalCleanComment"]))
plt.figure(figsize=(40, 30))
plt.imshow(cloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()"""

#Use For Fake
"""cloud = WordCloud(max_words = 18000,
                  width = 1600,
                  height = 800,
                  stopwords = STOPWORDS,
                  background_color = "white",
                  collocations=False).generate(" ".join(df["FinalCleanComment"]))
plt.figure(figsize=(40, 30))
plt.imshow(cloud, interpolation="bilinear")
plt.axis("off")
plt.tight_layout(pad=0)
plt.show()"""

# Export Result

In [30]:
df.to_csv("3.1ReviewsUpdatedDec19.csv", encoding='utf-8', index=False, mode='a')